In [ ]:
import pdfplumber
import pandas as pd
import re

# 1. Configuración de archivos
pdf_path = '../data/raw/China Tariffs_2025HTSRev32.pdf'  # El nombre exacto de tu archivo
output_excel = '../data/intermediate/Extraccion_China_Tariffs.xlsx'   # Nombre del archivo que se creará

print(f"Abriendo {pdf_path}...")

# Lista para guardar los datos encontrados
datos_extraidos = []

# 2. Definimos el patrón exacto que buscamos (Expresión Regular)
# Explicación del patrón:
# (\d{4}\.\d{2}\.\d{2})  -> Busca el código HTS (ej: 0101.21.00)
# \s+                    -> Busca cualquier espacio en medio
# (9903\.\d{2}\.\d{2})   -> Busca el código del Cap 99 (ej: 9903.88.15)
patron_regex = re.compile(r'(\d{4}\.\d{2}\.\d{2})\s+.*?(9903\.\d{2}\.\d{2})')

# 3. Proceso de lectura del PDF
with pdfplumber.open(pdf_path) as pdf:
    total_paginas = len(pdf.pages)
    print(f"El documento tiene {total_paginas} páginas.")

    for i, pagina in enumerate(pdf.pages):
        # Extraer todo el texto de la página
        texto = pagina.extract_text()

        if texto:
            # Buscar todas las coincidencias del patrón en esta página
            coincidencias = patron_regex.findall(texto)

            for match in coincidencias:
                hts_code, heading_99 = match
                datos_extraidos.append({
                    'HTS_Code': hts_code,
                    'Chapter_99_Heading': heading_99
                })

        # Mostrar progreso cada 10 páginas para que veas que avanza
        if (i + 1) % 50 == 0:
            print(f"Procesando página {i + 1} de {total_paginas}...")

# 4. Guardar en Excel
print(f"\nExtracción finalizada. Se encontraron {len(datos_extraidos)} códigos.")
print("Guardando en Excel...")

df = pd.DataFrame(datos_extraidos)

# Limpieza básica: eliminar duplicados si el PDF repite encabezados
df = df.drop_duplicates()

df.to_excel(output_excel, index=False)

print(f"¡Éxito! Archivo guardado como: {output_excel}")
print(df.head()) # Muestra las primeras 5 filas

Abriendo ../data/raw/China Tariffs_2025HTSRev32.pdf...
El documento tiene 257 páginas.
Procesando página 50 de 257...
Procesando página 100 de 257...
Procesando página 150 de 257...
Procesando página 200 de 257...
Procesando página 250 de 257...

Extracción finalizada. Se encontraron 10422 códigos.
Guardando en Excel...
¡Éxito! Archivo guardado como: ../data/intermediate/Extraccion_China_Raw.xlsx
     HTS_Code Chapter_99_Heading
0  9903.88.01         9903.88.02
1  9903.88.04         9903.88.15
2  0101.21.00         9903.88.15
3  0101.29.00         9903.88.15
4  0101.30.00         9903.88.15
